# Day 4 - Lab 2: Generating a CI/CD Pipeline (Solution)

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Introduction:**
This solution notebook provides the complete prompts for generating the `requirements.txt`, `Dockerfile`, and GitHub Actions workflow files. It demonstrates how to prompt for specific, structured configuration-as-code artifacts.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer, recommended_models_table

In [2]:
recommended_models_table()

| Model | Provider | Text | Vision | Image Gen | Image Edit | Audio Transcription | Context Window | Max Output Tokens |
|---|---|---|---|---|---|---|---|---|
| Qwen/Qwen-Image | huggingface | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |
| Qwen/Qwen-Image-Edit | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |
| black-forest-labs/FLUX.1-Kontext-dev | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |
| claude-opus-4-1-20250805 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| claude-opus-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| claude-sonnet-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |
| dall-e-3 | openai | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |
| deepseek-ai/DeepSeek-V3.1 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| gemini-1.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 8,192 |
| gemini-1.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 2,000,000 | 8,192 |
| gemini-2.0-flash-exp | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-preview-image-generation | google | ❌ | ❌ | ✅ | ❌ | ❌ | 32,000 | 8,192 |
| gemini-2.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-flash-image-preview | google | ❌ | ❌ | ✅ | ❌ | ❌ | 32,768 | 32,768 |
| gemini-2.5-flash-lite | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-live-2.5-flash-preview | google | ❌ | ❌ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |
| gpt-4.1 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,768 |
| gpt-4.1-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-nano | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4o | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-5-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-mini-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-nano-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| meta-llama/Llama-3.3-70B-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 8,192 | 4,096 |
| meta-llama/Llama-4-Maverick-17B-128E-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |
| meta-llama/Llama-4-Scout-17B-16E-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 10,000,000 | 100,000 |
| mistralai/Mistral-7B-Instruct-v0.3 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 32,768 | 8,192 |
| o3 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| o4-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| stabilityai/stable-diffusion-3.5-large | huggingface | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |
| tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| veo-3.0-fast-generate-preview | google | ❌ | ❌ | ❌ | ❌ | ❌ | 1,024 | - |
| veo-3.0-generate-preview | google | ❌ | ❌ | ❌ | ❌ | ❌ | 1,024 | - |
| whisper-1 | openai | ❌ | ❌ | ❌ | ❌ | ✅ | - | - |

'| Model | Provider | Text | Vision | Image Gen | Image Edit | Audio Transcription | Context Window | Max Output Tokens |\n|---|---|---|---|---|---|---|---|---|\n| Qwen/Qwen-Image | huggingface | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |\n| Qwen/Qwen-Image-Edit | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |\n| black-forest-labs/FLUX.1-Kontext-dev | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |\n| claude-opus-4-1-20250805 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-opus-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-sonnet-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |\n| dall-e-3 | openai | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |\n| deepseek-ai/DeepSeek-V3.1 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| gemini-1.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 8,192 |\n| gemini-1.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 2,000,000 | 8,192 |\n| gemini-2.0-flash-exp | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |\n| gemini-2.0-flash-preview

In [3]:
# Initialize different LLM clients for specialized CI/CD tasks
print("=== INITIALIZING SPECIALIZED LLM CLIENTS ===")

# Dependency analysis client - Fast model for simple parsing tasks
deps_client, deps_model_name, deps_api_provider = setup_llm_client(model_name="gemini-2.5-flash")
print(f"Dependencies analysis: {deps_model_name} ({deps_api_provider})")

# Docker configuration client - Balanced model for infrastructure as code
docker_client, docker_model_name, docker_api_provider = setup_llm_client(model_name="gpt-4o")
print(f"Docker configuration: {docker_model_name} ({docker_api_provider})")

# CI/CD workflow client - Advanced model for complex YAML generation
cicd_client, cicd_model_name, cicd_api_provider = setup_llm_client(model_name="deepseek-ai/DeepSeek-V3.1")
print(f"CI/CD workflows: {cicd_model_name} ({cicd_api_provider})")

print("✅ All specialized clients initialized successfully!")
print("=" * 50)

=== INITIALIZING SPECIALIZED LLM CLIENTS ===


/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-21 21:42:05,626 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-flash latency_ms=None artifacts_path=None


Dependencies analysis: gemini-2.5-flash (google)


2025-09-21 21:42:05,883 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None
2025-09-21 21:42:05,961 ag_aisoftdev.utils INFO LLM Client configured provider=huggingface model=deepseek-ai/DeepSeek-V3.1 latency_ms=None artifacts_path=None


Docker configuration: gpt-4o (openai)
CI/CD workflows: deepseek-ai/DeepSeek-V3.1 (huggingface)
✅ All specialized clients initialized successfully!


In [4]:
# Load the application code from Day 3 to provide context
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")
else:
    print(f"✅ Successfully loaded application code ({len(app_code)} characters)")
    print("First 200 characters preview:")
    print("-" * 50)
    print(app_code[:200] + "..." if len(app_code) > 200 else app_code)
    print("-" * 50)

✅ Successfully loaded application code (1277 characters)
First 200 characters preview:
--------------------------------------------------
from typing import List
from fastapi import Depends, FastAPI, HTTPException
from sqlalchemy.orm import Session
from pydantic import BaseModel
from app.db_models import SessionLocal, engine, Base, User...
--------------------------------------------------


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Explanation:**
This challenge demonstrates how prompt enhancement can transform a simple dependency analysis task into a sophisticated, production-ready solution. We start with a basic prompt for analyzing Python imports, but enhance it using our `prompt_enhancer` function to apply advanced prompt engineering techniques.

The `prompt_enhancer` automatically:
- Assigns the appropriate expert persona ("You are a Python Dependency Analysis Expert")
- Provides structured context and clear task definitions
- Sets explicit output format expectations
- Applies best practices for reliable, consistent results

We use a fast model (Gemini Flash) for this straightforward parsing task, demonstrating how different models can be optimized for different complexity levels. This approach is much faster and less error-prone than manually creating dependency files.

In [5]:
# Challenge 1: Enhanced Requirements Generation
print("=" * 60)
print("CHALLENGE 1: ENHANCED REQUIREMENTS GENERATION")
print("=" * 60)

if app_code:
    # Step 1: Create and enhance the raw prompt
    print("\n--- STEP 1: ENHANCING REQUIREMENTS PROMPT ---")
    raw_requirements_prompt = f"""Analyze the following Python code and generate a requirements.txt file listing all the external libraries imported.

Include versions for key libraries like fastapi, uvicorn, sqlalchemy, and pydantic. Also include pytest for running tests.

--- PYTHON CODE ---
{app_code}
--- END CODE ---

Output only the contents of the requirements.txt file."""

    enhanced_requirements_prompt = prompt_enhancer(
        raw_requirements_prompt, 
        deps_model_name, 
        deps_client, 
        deps_api_provider
    )
    print("Enhanced requirements prompt:")
    print("-" * 50)
    print(enhanced_requirements_prompt[:300] + "..." if len(enhanced_requirements_prompt) > 300 else enhanced_requirements_prompt)
    print("-" * 50)

    # Step 2: Generate requirements using specialized client
    print("\n--- STEP 2: GENERATING REQUIREMENTS ---")
    print(f"Using {deps_model_name} ({deps_api_provider}) for dependency analysis...")
    
    requirements_content = get_completion(
        enhanced_requirements_prompt, 
        deps_client, 
        deps_model_name, 
        deps_api_provider,
        temperature=0.2  # Lower temperature for consistent parsing
    )
    
    cleaned_reqs = clean_llm_output(requirements_content, language='text')
    print("Generated requirements.txt:")
    print("-" * 50)
    print(cleaned_reqs)
    print("-" * 50)
    
    save_artifact(cleaned_reqs, "requirements.txt")
    print("✅ Requirements.txt saved successfully!")
else:
    print("❌ Skipping requirements generation because app code is missing.")

print("\n" + "=" * 60)
print("CHALLENGE 1 COMPLETED!")
print("=" * 60)

CHALLENGE 1: ENHANCED REQUIREMENTS GENERATION

--- STEP 1: ENHANCING REQUIREMENTS PROMPT ---
Enhanced requirements prompt:
--------------------------------------------------
```xml
<persona>
You are an expert Python developer and a meticulous dependency management specialist. Your task is to analyze Python code and accurately generate a `requirements.txt` file, adhering to best practices for dependency specification.
</persona>

<context>
The user has provided a Python ...
--------------------------------------------------

--- STEP 2: GENERATING REQUIREMENTS ---
Using gemini-2.5-flash (google) for dependency analysis...
Generated requirements.txt:
--------------------------------------------------
fastapi==0.111.0
pydantic==2.7.1
pytest==8.2.0
sqlalchemy==2.0.30
uvicorn==0.29.0
--------------------------------------------------
✅ Requirements.txt saved successfully!

CHALLENGE 1 COMPLETED!


### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Explanation:**
This challenge showcases how prompt enhancement elevates infrastructure-as-code generation to professional standards. We transform a basic Dockerfile request into a comprehensive DevOps specification that incorporates security best practices, performance optimizations, and production-readiness guidelines.

The enhancement process ensures our prompt:
- Establishes expert DevOps persona with containerization specialization
- Explicitly requests security features (non-root user, multi-stage builds)
- Defines performance optimizations (slim base images, layer caching)
- Provides structured output requirements with clear technical specifications

We use GPT-4o for this intermediate task, as it requires balancing multiple concerns (security, performance, maintainability) while generating syntactically correct Dockerfile syntax. This represents the sweet spot between capability and cost-effectiveness for infrastructure generation.

In [6]:
# Challenge 2: Enhanced Dockerfile Generation
print("=" * 60)
print("CHALLENGE 2: ENHANCED DOCKERFILE GENERATION")
print("=" * 60)

# Step 1: Create and enhance the raw Dockerfile prompt
print("\n--- STEP 1: ENHANCING DOCKERFILE PROMPT ---")
raw_dockerfile_prompt = """Generate a best-practice, multi-stage Dockerfile for a production Python FastAPI application.

The Dockerfile must:
1. Use python:3.11-slim as the base image.
2. The first stage should install dependencies from requirements.txt.
3. The final stage should copy the application code and the installed dependencies from the first stage.
4. Expose port 8000.
5. The final CMD should run the application using uvicorn, binding to host 0.0.0.0.
6. Follow security best practices including running as non-root user.
7. Optimize for minimal image size and efficient layer caching.

Output only the raw Dockerfile content."""

enhanced_dockerfile_prompt = prompt_enhancer(
    raw_dockerfile_prompt, 
    docker_model_name, 
    docker_client, 
    docker_api_provider
)
print("Enhanced Dockerfile prompt:")
print("-" * 50)
print(enhanced_dockerfile_prompt[:400] + "..." if len(enhanced_dockerfile_prompt) > 400 else enhanced_dockerfile_prompt)
print("-" * 50)

# Step 2: Generate Dockerfile using specialized client
print("\n--- STEP 2: GENERATING DOCKERFILE ---")
print(f"Using {docker_model_name} ({docker_api_provider}) for Docker configuration...")

dockerfile_content = get_completion(
    enhanced_dockerfile_prompt, 
    docker_client, 
    docker_model_name, 
    docker_api_provider,
    temperature=0.3  # Slightly higher for creative infrastructure solutions
)

cleaned_dockerfile = clean_llm_output(dockerfile_content, language='dockerfile')
print("Generated Dockerfile:")
print("-" * 50)
print(cleaned_dockerfile)
print("-" * 50)

if cleaned_dockerfile:
    save_artifact(cleaned_dockerfile, "Dockerfile")
    print("✅ Dockerfile saved successfully!")
else:
    print("❌ Failed to generate valid Dockerfile content.")

print("\n" + "=" * 60)
print("CHALLENGE 2 COMPLETED!")
print("=" * 60)

CHALLENGE 2: ENHANCED DOCKERFILE GENERATION

--- STEP 1: ENHANCING DOCKERFILE PROMPT ---
Enhanced Dockerfile prompt:
--------------------------------------------------
```plaintext
<persona>
You are an expert Dockerfile developer specializing in creating efficient, secure, and production-ready Dockerfiles for Python applications.
</persona>

<context>
You need to create a multi-stage Dockerfile for a Python FastAPI application. The application is intended for production use, so it must be optimized for minimal image size and efficient layer caching. Security bes...
--------------------------------------------------

--- STEP 2: GENERATING DOCKERFILE ---
Using gpt-4o (openai) for Docker configuration...
Generated Dockerfile:
--------------------------------------------------
# First stage: Build
FROM python:3.11-slim AS builder

# Set working directory
WORKDIR /app

# Install build dependencies
RUN apt-get update && apt-get install -y --no-install-recommends gcc libffi-dev && \
    rm -

### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Explanation:**
This challenge represents the pinnacle of Configuration-as-Code generation, where we transform a basic CI/CD request into a comprehensive, production-ready automation pipeline. The prompt enhancement process applies sophisticated engineering principles to ensure the generated workflow adheres to industry best practices.

The enhancement incorporates:
- **Expert CI/CD Specialist Persona**: Establishes deep domain expertise in automation pipelines
- **Comprehensive Context Grounding**: Provides detailed understanding of GitHub Actions ecosystem
- **Chain-of-Thought Integration**: Guides the model through systematic workflow design decisions
- **Strict Output Validation**: Ensures syntactically correct YAML with proper schema compliance
- **Security and Performance Considerations**: Integrates caching, parallelization, and secure practices

We use DeepSeek-V3.1 for this advanced task because it excels at complex, multi-step reasoning required for sophisticated workflow orchestration. This model's architectural strengths in logical sequencing and dependency management make it ideal for generating robust CI/CD pipelines that can handle real-world complexity and edge cases.

In [7]:
# Challenge 3: Enhanced GitHub Actions Workflow Generation
print("=" * 60)
print("CHALLENGE 3: ENHANCED CI/CD WORKFLOW GENERATION")
print("=" * 60)

# Step 1: Create and enhance the raw CI/CD workflow prompt
print("\n--- STEP 1: ENHANCING CI/CD WORKFLOW PROMPT ---")
raw_cicd_prompt = """Generate a complete GitHub Actions workflow file named ci.yml for a Python FastAPI project.

The workflow must:
- Be named 'Build and Test'.
- Trigger on every push to the main branch and on pull requests.
- Define one job named build-and-test that runs on ubuntu-latest.
- The job must have the following sequential steps:
  1. actions/checkout@v4 to check out the repository code.
  2. actions/setup-python@v5 to set up Python 3.11.
  3. A step to cache pip dependencies for faster builds.
  4. A step to install dependencies using pip from requirements.txt.
  5. A step to run the test suite using pytest with verbose output.
  6. A step to run basic linting/formatting checks.

Include proper error handling, environment variables, and follow GitHub Actions best practices.
Output only the raw YAML content for the file."""

enhanced_cicd_prompt = prompt_enhancer(
    raw_cicd_prompt, 
    cicd_model_name, 
    cicd_client, 
    cicd_api_provider
)
print("Enhanced CI/CD workflow prompt:")
print("-" * 50)
print(enhanced_cicd_prompt[:500] + "..." if len(enhanced_cicd_prompt) > 500 else enhanced_cicd_prompt)
print("-" * 50)

# Step 2: Generate CI/CD workflow using specialized client
print("\n--- STEP 2: GENERATING CI/CD WORKFLOW ---")
print(f"Using {cicd_model_name} ({cicd_api_provider}) for advanced workflow orchestration...")

ci_workflow_content = get_completion(
    enhanced_cicd_prompt, 
    cicd_client, 
    cicd_model_name, 
    cicd_api_provider,
    temperature=0.2  # Low temperature for consistent, reliable automation
)

cleaned_yaml = clean_llm_output(ci_workflow_content, language='yaml')
print("Generated GitHub Actions Workflow:")
print("-" * 50)
print(cleaned_yaml)
print("-" * 50)

if cleaned_yaml:
    save_artifact(cleaned_yaml, ".github/workflows/ci.yml")
    print("✅ GitHub Actions workflow saved successfully!")
else:
    print("❌ Failed to generate valid workflow content.")

print("\n" + "=" * 60)
print("CHALLENGE 3 COMPLETED!")
print("=" * 60)

CHALLENGE 3: ENHANCED CI/CD WORKFLOW GENERATION

--- STEP 1: ENHANCING CI/CD WORKFLOW PROMPT ---
Enhanced CI/CD workflow prompt:
--------------------------------------------------
<persona>
You are a Senior DevOps Engineer and GitHub Actions specialist with deep expertise in creating robust, efficient, and secure CI/CD pipelines for Python applications. Your knowledge of caching strategies, dependency management, and testing best practices is authoritative.
</persona>

<context>
The task is to generate a complete GitHub Actions workflow YAML file for a Python FastAPI project. The workflow must automate the processes of building, testing, and performing basic code quality ...
--------------------------------------------------

--- STEP 2: GENERATING CI/CD WORKFLOW ---
Using deepseek-ai/DeepSeek-V3.1 (huggingface) for advanced workflow orchestration...
Generated GitHub Actions Workflow:
--------------------------------------------------
name: Python CI

on:
  push:
    branches: [ main ]

## Lab Conclusion

Outstanding! You have successfully demonstrated the power of **Enhanced AI-Driven DevOps** by generating a complete, enterprise-grade Continuous Integration pipeline using advanced prompt engineering techniques and strategic model selection.

### Key Innovations Demonstrated:

1. **Prompt Enhancement Engineering**: Transformed basic requests into sophisticated, production-ready specifications using our `prompt_enhancer` meta-system that applies expert personas, contextual grounding, and structured output requirements.

2. **Strategic Model Selection**: 
   - **Gemini Flash** for fast dependency parsing (foundational tasks)
   - **GPT-4o** for balanced infrastructure generation (intermediate complexity)
   - **DeepSeek-V3.1** for complex workflow orchestration (advanced reasoning)

3. **Configuration-as-Code Mastery**: Generated three critical DevOps artifacts:
   - `requirements.txt` (dependency management)
   - `Dockerfile` (containerization with security best practices)
   - `ci.yml` (automated testing and integration pipeline)

### Professional Impact:

This lab demonstrates how AI can elevate DevOps practices from manual, error-prone processes to automated, consistent, and secure infrastructure generation. By combining prompt enhancement with strategic model selection, you've created a scalable approach that can be applied across different CI/CD complexity levels while maintaining cost-effectiveness and quality.

> **Key Takeaway:** The integration of prompt enhancement with specialized model selection represents the next evolution in AI-assisted DevOps. This approach enables teams to generate sophisticated, production-ready infrastructure configurations that incorporate security best practices, performance optimizations, and industry standards—all while maintaining the flexibility to optimize for different complexity levels and cost considerations.